In [5]:
import numpy as np
import json

In [6]:
dataset = "symmetric"

In [34]:
path = "/home/xianglin/data/noisy/{}/embedding.npy".format(dataset)
embeddings = np.load(path)

path = "/home/xianglin/data/noisy/{}/clean_label.json".format(dataset)
with open(path, "r") as f:
    clean_label = json.load(f)
path = "/home/xianglin/data/noisy/{}/noisy_label.json".format(dataset)
with open(path, "r") as f:
    noisy_label = json.load(f)

clean_label = np.array(clean_label)
noisy_label = np.array(noisy_label)

In [22]:
import os, sys
import numpy as np
import torch
import json

sys.path.append("/home/xianglin/git_space/DLVisDebugger")
from singleVis.SingleVisualizationModel import SingleVisualizationModel
from singleVis.data import NormalDataProvider
from singleVis.eval.evaluator import Evaluator
from singleVis.projector import Projector

/home/xianglin/miniconda3/envs/timevis/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
CONTENT_PATH = "/home/xianglin/data/noisy/symmetric"
sys.path.append(CONTENT_PATH)
from config import config

In [24]:
GPU_ID = 0
SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
DATASET = config["DATASET"]
PREPROCESS = config["VISUALIZATION"]["PREPROCESS"]
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]

# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
LEN = TRAINING_PARAMETER["train_num"]

# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
LAMBDA = VISUALIZATION_PARAMETER["LAMBDA"]
S_LAMBDA = VISUALIZATION_PARAMETER["S_LAMBDA"]
B_N_EPOCHS = VISUALIZATION_PARAMETER["BOUNDARY"]["B_N_EPOCHS"]
L_BOUND = VISUALIZATION_PARAMETER["BOUNDARY"]["L_BOUND"]
INIT_NUM = VISUALIZATION_PARAMETER["INIT_NUM"]
ALPHA = VISUALIZATION_PARAMETER["ALPHA"]
BETA = VISUALIZATION_PARAMETER["BETA"]
MAX_HAUSDORFF = VISUALIZATION_PARAMETER["MAX_HAUSDORFF"]
HIDDEN_LAYER = VISUALIZATION_PARAMETER["HIDDEN_LAYER"]
S_N_EPOCHS = VISUALIZATION_PARAMETER["S_N_EPOCHS"]
T_N_EPOCHS = VISUALIZATION_PARAMETER["T_N_EPOCHS"]
N_NEIGHBORS = VISUALIZATION_PARAMETER["N_NEIGHBORS"]
PATIENT = VISUALIZATION_PARAMETER["PATIENT"]
MAX_EPOCH = VISUALIZATION_PARAMETER["MAX_EPOCH"]
SEGMENTS = VISUALIZATION_PARAMETER["SEGMENTS"]
RESUME_SEG = VISUALIZATION_PARAMETER["RESUME_SEG"]
# define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")

content_path = CONTENT_PATH
sys.path.append(content_path)

import Model.model as subject_model
# net = resnet18()
net = eval("subject_model.{}()".format(NET))
classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")
data_provider = NormalDataProvider(CONTENT_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, classes=CLASSES,verbose=1)
if PREPROCESS:
    data_provider.initialize(LEN//10, l_bound=L_BOUND)

model = SingleVisualizationModel(input_dims=512, output_dims=2, units=256, hidden_layer=HIDDEN_LAYER)
projector = Projector(vis_model=model, content_path=CONTENT_PATH, segments=SEGMENTS, device=DEVICE)

Finish initialization...


In [27]:
data = data_provider.train_representation(200)
pred = data_provider.get_pred(200, data)
predictions = pred.argmax(1)

100%|██████████| 250/250 [00:00<00:00, 439.56it/s]


In [ ]:
dataset = "pairflip"
path = "/home/xianglin/projects/DVI_data/resnet18_cifar10/embedding.npy"
embeddings = np.load(path)
path = "/home/xianglin/projects/DVI_data/noisy/{}/cifar10/clean_label.json".format(dataset)
with open(path, "r") as f:
    clean_label = json.load(f)
    
clean_label = np.array(clean_label)
noisy_label = np.copy(clean_label)

In [35]:
embeddings = embeddings.reshape(50000,-1,2)
embeddings.shape

(50000, 200, 2)

In [36]:
embeddings = embeddings[:,80:,:]

In [37]:
v = embeddings[:,1:,:]-embeddings[:,:-1,:]
a = v[:,1:,:]-v[:,:-1,:]
v.shape, a.shape

((50000, 119, 2), (50000, 118, 2))

In [55]:
from sklearn import metrics
from sklearn.cluster import Birch
brc = Birch(n_clusters=30)
brc.fit(a.reshape(50000, -1))

Birch(n_clusters=30)

In [56]:
c = 30
gt = np.argwhere(noisy_label!=clean_label).squeeze()
rates = np.zeros(c)
wrong = np.zeros(c)
sel = np.zeros(c)
for suspect in range(c):
    selected_idxs = np.argwhere(brc.labels_==suspect).squeeze()
    wrong[suspect] = len(np.intersect1d(selected_idxs, gt))
    sel[suspect] = len(selected_idxs)
    rates[suspect] = len(np.intersect1d(selected_idxs, gt))/len(selected_idxs)

In [57]:
ranking = np.argsort(rates)
for r in range(c-1,-1,-1):
    print(round(rates[ranking[r]], 3),"\t", int(wrong[ranking[r]]),"/", int(sel[ranking[r]]))

0.493 	 414 / 840
0.481 	 235 / 489
0.423 	 263 / 622
0.39 	 99 / 254
0.383 	 70 / 183
0.316 	 279 / 883
0.3 	 159 / 530
0.299 	 408 / 1365
0.269 	 98 / 364
0.269 	 175 / 651
0.267 	 236 / 883
0.26 	 368 / 1414
0.255 	 208 / 817
0.242 	 145 / 599
0.236 	 213 / 903
0.212 	 155 / 730
0.207 	 150 / 725
0.198 	 264 / 1334
0.197 	 378 / 1915
0.193 	 180 / 935
0.187 	 117 / 626
0.173 	 4225 / 24449
0.16 	 92 / 576
0.151 	 322 / 2132
0.14 	 221 / 1574
0.122 	 92 / 755
0.122 	 47 / 386
0.111 	 257 / 2312
0.105 	 44 / 418
0.089 	 30 / 336


In [ ]:
predict_labels = brc.labels_
suspect = np.argmin(np.bincount(predict_labels))
selected_idxs = np.argwhere(predictions==suspect).squeeze()
gt = np.argwhere(noisy_label!=clean_label).squeeze()
len(np.intersect1d(selected_idxs, gt)), len(selected_idxs)

In [15]:
brc = Birch(n_clusters=20)
brc.fit(v.reshape(50000, -1))

Birch(n_clusters=20)

In [16]:
predict_labels = brc.labels_
suspect = np.argmin(np.bincount(predict_labels))
selected_idxs = np.argwhere(brc.labels_==suspect).squeeze()
gt = np.argwhere(noisy_label!=clean_label).squeeze()
len(np.intersect1d(selected_idxs, gt)), len(selected_idxs)

(114, 257)

In [20]:
brc = Birch(n_clusters=20)
data = np.concatenate((a.reshape(50000, -1), v.reshape(50000,-1)), axis=1)
brc.fit(data)

Birch(n_clusters=20)

In [21]:
predict_labels = brc.labels_
suspect = np.argmin(np.bincount(predict_labels))
selected_idxs = np.argwhere(brc.labels_==suspect).squeeze()
gt = np.argwhere(noisy_label!=clean_label).squeeze()
len(np.intersect1d(selected_idxs, gt)), len(selected_idxs)

(110, 328)

In [ ]:
from noise_detector import NoiseTrajectoryDetector
ntd = NoiseTrajectoryDetector(embeddings, noisy_label)

In [ ]:
# cls_num=2
# ntd.proj_cls(cls_num, period=75, repeat=2)
ntd.proj_all(period=75) # 75

In [ ]:
for cls_num in range(10):
    # print(cls_num, ntd.detect_noise_cls(cls_num))
    best_s, best_c = ntd.trajectory_eval[str(cls_num)]
    print("silhouette_score\t", best_s)
    print("calinski_harabasz_score\t", best_c)
    flag = best_c>.5

    print(cls_num, flag)

In [ ]:
for cls_num in range(10):
    # print(cls_num, ntd.detect_noise_cls(cls_num))
    best_s, best_c = ntd.trajectory_eval[str(cls_num)]
    print("silhouette_score\t", best_s)
    print("calinski_harabasz_score\t", best_c)
    flag = best_c>.5
    print(cls_num, flag)

In [ ]:
for num in range(10):
    ntd.show_ground_truth(num,clean_label[noisy_label==num])
    ntd.show_scores(num)

In [ ]:
for num in range(10):
    ntd.show_ground_truth(num,clean_label[noisy_label==num])
    ntd.show_scores(num)

In [ ]:
from noise_detector import select_closest
import math

for num in range(10):
    print("=================================={}=========================================".format(num))
    clean_labels = clean_label[noisy_label==num]
    ntd.show_ground_truth(num,clean_labels)

    c = ntd.sub_centers[str(num)]
    embedding = ntd.trajectory_embedding[str(num)]
    centroids = embedding[c]

    # vote for labels and scores
    c_labels = select_closest(embedding, centroids)
    centroid_labels = np.zeros(len(centroids))
    for i in range(len(centroids)):
        centroid_labels[i] = np.bincount(clean_labels[c_labels==i]).argmax()

    gt = centroid_labels
    nt = np.sum(gt!=num)
    ct = len(c) - nt

    curr_nt = nt
    flag = math.floor(nt*0.9)
    target_flag = flag

    for t in range(len(c)):
        id, _, selected = ntd.suggest_abnormal(num)
        d = False
        # if clean_label[noisy_label==num][id] != num:ß
        if gt[id] != num:
            d = True
            flag=flag-1
            curr_nt=curr_nt-1
        ntd.update_belief(num, selected, d)
        # ntd.show_verified(num)
        if flag==0:
            print("[{}]\t{} rounds find {} (90%) noise samples".format(target_flag/(t+1), t+1, target_flag))
            flag=10000
        if curr_nt==0:
            print("[{}]\t{} rounds find all {} noise samples".format(nt/(t+1), t+1, len(c)-ct))
            break

In [ ]:
from noise_detector import select_closest
import math

# Ground Truth
for num in range(10):
    print("=================================={}=========================================".format(num))
    clean_labels = clean_label[noisy_label==num]
    ntd.show_ground_truth(num,clean_labels)

    c = ntd.sub_centers[str(num)]
    embedding = ntd.trajectory_embedding[str(num)]
    centroids = embedding[c]

    # vote for labels and scores
    c_labels = select_closest(embedding, centroids)
    centroid_labels = np.zeros(len(centroids))
    for i in range(len(centroids)):
        centroid_labels[i] = np.bincount(clean_labels[c_labels==i]).argmax()

    gt = centroid_labels
    nt = np.sum(gt!=num)
    ct = len(c) - nt

    curr_nt = nt
    flag = math.floor(nt*0.9)
    target_flag = flag

    candidate_list = np.arange(len(c)).tolist()

    for t in range(len(c)):
        # id, _, selected = ntd.suggest_abnormal(num)
        id = np.random.choice(candidate_list,1)[0]
        selected = centroids[id]
        candidate_list.remove(id)
        d = False
        # if clean_label[noisy_label==num][id] != num:
        if gt[id] != num:
            d = True
            flag=flag-1
            curr_nt=curr_nt-1
        ntd.update_belief(num, selected, d)
        if flag==0:
            print("[{}]\t{} rounds find {} (90%) noise samples".format(target_flag/(t+1), t+1, target_flag))
            flag=10000
        if curr_nt==0:
            print("[{}]\t{} rounds find all {} noise samples".format(nt/(t+1), t+1, len(c)-ct))
            break